### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import logging
import datetime, time
import platform
import itertools
# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import matplotlib.pyplot as plt

# OpenTNSim
import opentnsim

# Used for mathematical functions
import math             
import tqdm
# Used for making the graph to visualize our problem
import networkx as nx   

import plotly.express as px
from plotly.subplots import make_subplots

#logging.basicConfig(level=logging.DEBUG) #turn on all the debug messages
logging.basicConfig(level=logging.INFO)# turn off all the debug messages


### Create vessel - add VesselProperties and ConsumesEnergy mixins


In [2]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.energy.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  # needed to calculate resistances

In [3]:
# Create a dict with all important settings

data_vessel_up = {
    "env": None,
    "name": 'Vessel',
    "route": None,
    "geometry": None,
    "V_g_ave": None,  # m/s
    "type": None,
    "B": 9.5,
    "L": 85,
    "H_e": None, 
    "H_f": None, 
    "T": 2,
    "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": True, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Tanker", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "V_g_profile":None, # if use the V_g_profile to determine V_w, set to True, otherwise (use the V_g_ave to determine V_w) set to False.
    "P_tot_given_profile":False,
    "P_installed": 780, # kW  
    "P_tot_given": True, # kW
    "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "sailing_on_power": True,
    "sailing_upstream":False,
    "wind_influence": False, # if consider wind influence, set to True; otherwise set to False.
    "P_hotel_perc": 0,
    "P_hotel": None, # None: calculate P_hotel from percentage
    "x": 1,# number of propellers
    "L_w": 2.0 ,
    "C_B":0.9, 
    "C_year": 1961,
}             


In [4]:
# Create a dict with all important settings

data_vessel_down = {
    "env": None,
    "name": 'Vessel',
    "route": None,
    "geometry": None,
    "V_g_ave": None,  # m/s
    "type": None,
    "B": 9.5,
    "L": 85,
    "H_e": None, 
    "H_f": None, 
    "T": 2,
    "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": True, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Tanker", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "V_g_profile":None, # if use the V_g_profile to determine V_w, set to True, otherwise (use the V_g_ave to determine V_w) set to False.
    "P_tot_given_profile":False,
    "P_installed": 780, # kW  
    "P_tot_given": True, # kW
    "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "sailing_on_power": True,
    "sailing_upstream":False,
    "wind_influence": False, # if consider wind influence, set to True; otherwise set to False.
    "P_hotel_perc": 0,
    "P_hotel": None, # None: calculate P_hotel from percentage
    "x": 1,# number of propellers
    "L_w": 2.0 ,
    "C_B":0.9, 
    "C_year": 1961,
}             


### arrange input table

In [5]:
stretch = ["s1","s2","s3","s4","s5","s6","s7","s8",
          "s9","s10","s11","s12","s13","s14","s15"]
width = [150]
depth = [7.5, 4.18,7.83,6.5,10.6,6.45,8.04,5.65,8.87,4.93,7.7,9.8,5.45]
power_applied_down = [500,450,400,350,300,250,230,200,150,130,100,90]
power_applied_up = [620,250]

In [6]:
# prepare the work to be done by creating a list of all combinations
work_up = list(itertools.product(stretch, depth, width,power_applied_up))

# prepare a list of dictionaries for pandas
rows_up = []
for item in work_up:
    row_up = {"stretch": item[0],"depth": item[1],"width":item[2], "power_applied_up":item[3]}
    rows_up.append(row_up)

    # these are all the simulations that we want to run

# convert them to dataframe, so that we can apply a function and monitor progress
work_up_df = pd.DataFrame(rows_up)
work_up_df.tail(100)

,stretch,depth,width,power_applied_up
290,s12,7.83,150,620
291,s12,7.83,150,250
292,s12,6.50,150,620
293,s12,6.50,150,250
294,s12,10.60,150,620
...,...,...,...,...
385,s15,7.70,150,250
386,s15,9.80,150,620
387,s15,9.80,150,250
388,s15,5.45,150,620


### Run simulation


In [7]:
results_up = []

for i, row_up in tqdm.tqdm(work_up_df.iterrows(),disable=True):

    # get vessel 
    data_vessel_i = data_vessel_up.copy()
    vessel = TransportResource(**data_vessel_i)
    vessel.P_tot_given = row_up['power_applied_up']
    
    stretch = row_up['stretch']
    # estimate 'grounding speed' as a useful upperbound
    # upperbound, selected, results_df = opentnsim.strategy.get_upperbound_for_power2v(vessel, width=row['width'], depth=row['depth'], margin=0.3,bounds=(0, 20))
    # print(upperbound)
    # calculate the velocity that belongs to the T_strategy (while leaving the margin)
    V_w_up = opentnsim.strategy.power2v(vessel, h_0=row_up['depth'],power_applied=row_up['power_applied_up'], upperbound=5)
    # V_w_down = opentnsim.strategy.power2v(vessel, h_0=row['depth'],power_applied=row['power_applied_down'], upperbound=5)
    
    result_up ={}
    result_up.update(row_up)
    
    result_up['V_w_up (m/s)'] = V_w_up    
    result_up['V_w_up (km/h)'] = V_w_up * 3.6
    # result_up['V_w_down (m/s)'] = V_w_down    
    # result_up['V_w_down (km/h)'] = V_w_down * 3.6
    results_up.append(result_up)

In [8]:
results_up_df = pd.DataFrame(results_up)
results_up_df

,stretch,depth,width,power_applied_up,V_w_up (m/s),V_w_up (km/h)
0,s1,7.50,150,620,4.426327,15.934776
1,s1,7.50,150,250,3.281770,11.814370
2,s1,4.18,150,620,3.732659,13.437573
3,s1,4.18,150,250,3.046009,10.965632
4,s1,7.83,150,620,4.219130,15.188869
...,...,...,...,...,...,...
385,s15,7.70,150,250,3.282033,11.815319
386,s15,9.80,150,620,4.450685,16.022467
387,s15,9.80,150,250,3.750199,13.500718
388,s15,5.45,150,620,4.423426,15.924334


In [9]:
V1_up = results_up_df.query('stretch == "s1" & depth == 7.5 & power_applied_up == 620')
V2_up = results_up_df.query('stretch == "s2" & depth == 4.18 & power_applied_up == 620')
V3_up = results_up_df.query('stretch == "s3" & depth == 4.18 & power_applied_up == 620')
V4_up = results_up_df.query('stretch == "s4" & depth == 7.83 & power_applied_up == 620')
V5_up = results_up_df.query('stretch == "s5" & depth == 7.5 & power_applied_up == 620')
V6_up = results_up_df.query('stretch == "s6" & depth == 6.5 & power_applied_up == 620')
V7_up = results_up_df.query('stretch == "s7" & depth == 10.6 & power_applied_up == 620')
V8_up = results_up_df.query('stretch == "s8" & depth == 6.45 & power_applied_up == 620')
V9_up = results_up_df.query('stretch == "s9" & depth == 8.04 & power_applied_up == 620')
V10_up = results_up_df.query('stretch == "s10" & depth == 5.65 & power_applied_up == 620')
V11_up = results_up_df.query('stretch == "s11" & depth == 8.87 & power_applied_up == 620')
V12_up = results_up_df.query('stretch == "s12" & depth == 4.93 & power_applied_up == 620')
V13_up = results_up_df.query('stretch == "s13" & depth == 7.7 & power_applied_up == 620')
V14_up = results_up_df.query('stretch == "s14" & depth == 9.8 & power_applied_up == 250')
V15_up = results_up_df.query('stretch == "s15" & depth == 5.45 & power_applied_up == 620')

V_up = pd.concat([V1_up,V2_up,V3_up,V4_up,V5_up,V6_up,V7_up,V8_up,V9_up,V10_up,V11_up,V12_up,V13_up,V14_up,V15_up])

V_up= V_up.drop_duplicates()
V_up.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
V_up

,stretch,depth,width,power_applied_up,V_w_up (m/s),V_w_up (km/h)
1,s1,7.50,150,620,4.426327,15.934776
2,s2,4.18,150,620,3.732659,13.437573
3,s3,4.18,150,620,3.732659,13.437573
4,s4,7.83,150,620,4.219130,15.188869
5,s5,7.50,150,620,4.426327,15.934776
6,s6,6.50,150,620,4.425300,15.931079
7,s7,10.60,150,620,4.450890,16.023203
8,s8,6.45,150,620,4.425234,15.930842
9,s9,8.04,150,620,4.219258,15.189330
10,s10,5.65,150,620,4.423883,15.925980


In [10]:
stretch = ["s1","s2","s3","s4","s5","s6","s7","s8",
          "s9","s10","s11","s12","s13","s14","s15"]
width = [150]
depth = [7.5, 4.18,7.83,6.5,10.6,6.45,8.04,5.65,8.87,4.93,7.7,9.8,5.45]
power_applied_down = [500,450,400,350,300,250,230,200,150,130,100,90]
power_applied_up = [620,400]

In [11]:
# prepare the work to be done by creating a list of all combinations
work_down = list(itertools.product(stretch, depth, width, power_applied_down))

# prepare a list of dictionaries for pandas
rows_down = []
for item in work_down:
    row_down = {"stretch": item[0], "depth": item[1],"width":item[2], "power_applied_down": item[3]}
    rows_down.append(row_down)

    # these are all the simulations that we want to run

# convert them to dataframe, so that we can apply a function and monitor progress
work_down_df = pd.DataFrame(rows_down)
work_down_df.tail(100)

,stretch,depth,width,power_applied_down
2240,s15,10.60,150,150
2241,s15,10.60,150,130
2242,s15,10.60,150,100
2243,s15,10.60,150,90
2244,s15,6.45,150,500
...,...,...,...,...
2335,s15,5.45,150,200
2336,s15,5.45,150,150
2337,s15,5.45,150,130
2338,s15,5.45,150,100


In [12]:
results_down = []

for i, row_down in tqdm.tqdm(work_down_df.iterrows(),disable=True):

    # get vessel 
    data_vessel_i = data_vessel_down.copy()
    vessel = TransportResource(**data_vessel_i)
    vessel.P_tot_given = row_down['power_applied_down']
    stretch = row_down['stretch']
    # estimate 'grounding speed' as a useful upperbound
    # upperbound, selected, results_df = opentnsim.strategy.get_upperbound_for_power2v(vessel, width=row['width'], depth=row['depth'], margin=0.3,bounds=(0, 20))
    # print(upperbound)
    # calculate the velocity that belongs to the T_strategy (while leaving the margin)
    # V_w_up = opentnsim.strategy.power2v(vessel, h_0=row['depth'],power_applied=row['power_applied_up'], upperbound=5)
    V_w_down = opentnsim.strategy.power2v(vessel, h_0=row_down['depth'],power_applied=row_down['power_applied_down'], upperbound=5)
    
    result_down ={}
    result_down.update(row_down)
    
    # result['V_w_up (m/s)'] = V_w_up    
    # result['V_w_up (km/h)'] = V_w_up * 3.6
    result_down['V_w_down (m/s)'] = V_w_down    
    result_down['V_w_down (km/h)'] = V_w_down * 3.6
    results_down.append(result_down)

In [13]:
results_down_df = pd.DataFrame(results_down)
results_down_df

,stretch,depth,width,power_applied_down,V_w_down (m/s),V_w_down (km/h)
0,s1,7.50,150,500,3.845878,13.845160
1,s1,7.50,150,450,3.712852,13.366268
2,s1,7.50,150,400,3.832256,13.796121
3,s1,7.50,150,350,3.528733,12.703437
4,s1,7.50,150,300,3.415700,12.296521
...,...,...,...,...,...,...
2335,s15,5.45,150,200,3.110853,11.199070
2336,s15,5.45,150,150,2.892799,10.414078
2337,s15,5.45,150,130,2.782758,10.017929
2338,s15,5.45,150,100,2.558281,9.209812


In [14]:
V1_down = results_down_df.query('stretch == "s1" & depth == 7.5 & power_applied_down == 500')
V2_down = results_down_df.query('stretch == "s2" & depth == 4.18 & power_applied_down == 450')
V3_down = results_down_df.query('stretch == "s3" & depth == 4.18 & power_applied_down == 250')
V4_down = results_down_df.query('stretch == "s4" & depth == 7.83 & power_applied_down == 400')
V5_down = results_down_df.query('stretch == "s5" & depth == 7.5 & power_applied_down == 350')
V6_down = results_down_df.query('stretch == "s6" & depth == 6.5 & power_applied_down == 250')
V7_down = results_down_df.query('stretch == "s7" & depth == 10.6 & power_applied_down == 300')
V8_down = results_down_df.query('stretch == "s8" & depth == 6.45 & power_applied_down == 450')
V9_down = results_down_df.query('stretch == "s9" & depth == 8.04 & power_applied_down == 400')
V10_down = results_down_df.query('stretch == "s10" & depth == 5.65 & power_applied_down == 150')
V11_down = results_down_df.query('stretch == "s11" & depth == 8.87 & power_applied_down == 400')
V12_down = results_down_df.query('stretch == "s12" & depth == 4.93 & power_applied_down == 130')
V13_down = results_down_df.query('stretch == "s13" & depth == 7.7 & power_applied_down == 300')
V14_down = results_down_df.query('stretch == "s14" & depth == 9.8 & power_applied_down == 90')
V15_down = results_down_df.query('stretch == "s15" & depth == 5.45 & power_applied_down == 230')
V_down = pd.concat([V1_down,V2_down,V3_down,V4_down,V5_down,V6_down,
                    V7_down,V8_down,V9_down,V10_down,V11_down,V12_down,V13_down,V14_down,V15_down])
V_down= V_down.drop_duplicates()

V_down

,stretch,depth,width,power_applied_down,V_w_down (m/s),V_w_down (km/h)
0,s1,7.50,150,500,3.845878,13.845160
169,s2,4.18,150,450,3.483005,12.538817
329,s3,4.18,150,250,3.046009,10.965632
494,s4,7.83,150,400,3.625299,13.051077
627,s5,7.50,150,350,3.528733,12.703437
821,s6,6.50,150,250,3.280048,11.808174
988,s7,10.60,150,300,3.886111,13.990000
1153,s8,6.45,150,450,4.168708,15.007351
1322,s9,8.04,150,400,3.625500,13.051800
1496,s10,5.65,150,150,2.893693,10.417295


### get vessel velocity to the ground: upstream, downstream

In [15]:
current_speeds = pd.DataFrame([-2.24,-3.4,-4.93,-2.59,-3.26,-3.85,-1.87,-1.08,-2.83,-4.99,-2.89,-6.4,-3.42,-1.64,-4.1])
current_speeds.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
current_speeds.columns =['current_speeds (km/h)']

stretch_length = pd.DataFrame([48.28,46.02,32.23,18.87,27.4,19.2,37,15,94,30,87,48,21,37.56,171])
stretch_length.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
stretch_length.columns =['stretch_length (km)']
V_down.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
V_up = V_up.drop(["stretch","width"], axis=1)
V_down = V_down.drop(["stretch","depth","width"], axis=1)
V_df = pd.concat([stretch_length,current_speeds,V_up,V_down], axis = 1)
V_df['V_g_upstream (km/h)']= V_df['V_w_up (km/h)']+V_df['current_speeds (km/h)']
V_df['V_g_downstream (km/h)']= V_df['V_w_down (km/h)']-V_df['current_speeds (km/h)']
V_df['duration_upstream (h)']= V_df['stretch_length (km)']/V_df['V_g_upstream (km/h)']
V_df['duration_downstream (h)']= V_df['stretch_length (km)']/V_df['V_g_downstream (km/h)']

V_df

,stretch_length (km),current_speeds (km/h),depth,power_applied_up,V_w_up (m/s),V_w_up (km/h),power_applied_down,V_w_down (m/s),V_w_down (km/h),V_g_upstream (km/h),V_g_downstream (km/h),duration_upstream (h),duration_downstream (h)
1,48.28,-2.24,7.50,620,4.426327,15.934776,500,3.845878,13.845160,13.694776,16.085160,3.525432,3.001524
2,46.02,-3.40,4.18,620,3.732659,13.437573,450,3.483005,12.538817,10.037573,15.938817,4.584774,2.887291
3,32.23,-4.93,4.18,620,3.732659,13.437573,250,3.046009,10.965632,8.507573,15.895632,3.788389,2.027601
4,18.87,-2.59,7.83,620,4.219130,15.188869,400,3.625299,13.051077,12.598869,15.641077,1.497754,1.206439
5,27.40,-3.26,7.50,620,4.426327,15.934776,350,3.528733,12.703437,12.674776,15.963437,2.161774,1.716422
6,19.20,-3.85,6.50,620,4.425300,15.931079,250,3.280048,11.808174,12.081079,15.658174,1.589262,1.226197
7,37.00,-1.87,10.60,620,4.450890,16.023203,300,3.886111,13.990000,14.153203,15.860000,2.614249,2.332913
8,15.00,-1.08,6.45,620,4.425234,15.930842,450,4.168708,15.007351,14.850842,16.087351,1.010044,0.932410
9,94.00,-2.83,8.04,620,4.219258,15.189330,400,3.625500,13.051800,12.359330,15.881800,7.605591,5.918725
10,30.00,-4.99,5.65,620,4.423883,15.925980,150,2.893693,10.417295,10.935980,15.407295,2.743238,1.947130


### get vessel sailing duration: upstream, downstream, roundtrip

In [16]:
upstream_total_duration = V_df['duration_upstream (h)'].sum()
upstream_total_duration

65.06108747978257

In [17]:
downstream_total_duration = V_df['duration_downstream (h)'].sum()
downstream_total_duration

46.99871086948224